# RAG settings and Pinecone data uploaded

In [15]:
cd ..

/home/jorge/DocumentsWLS/Data_Science_Projects/Geospatial-similarity-combining-TDA-LLM-agents-and-MLOps-workflows


In [16]:
from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# === Claves principales ===
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# === LangSmith config ===
LANGSMITH_TRACING_V2 = os.getenv("LANGSMITH_TRACING_V2", "true")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT", "RAG-Municipios")

os.environ["LANGSMITH_TRACING_V2"] = LANGSMITH_TRACING_V2
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"


In [17]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
# Extract text from PDF files:

def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [20]:
extracted_data = load_pdf_files("flask_app/data_RAG")

In [21]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objetcs, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(page_content=doc.page_content,
            metadata={"source": src})
        )
    return minimal_docs

In [22]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500,
        chunk_overlap = 100,
        length_function=len
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

texts_chunk = text_split(minimal_docs)

print(f"Split blog post into {len(texts_chunk)} sub-documents.")

Split blog post into 26 sub-documents.


In [45]:
from langchain_text_splitters import TokenTextSplitter

def text_split_token(minimal_docs):
    text_splitter = TokenTextSplitter(
        chunk_size=500,         # número de tokens por chunk
        chunk_overlap=50,       # superposición entre chunks
        encoding_name="cl100k_base"  # tokenizer compatible con GPT-5-nano / GPT-4-turbo
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

texts_chunk = text_split_token(minimal_docs)

print(f"Split blog post into {len(texts_chunk)} sub-documents.")

Split blog post into 23 sub-documents.


In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
import torch
def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-distilroberta-v1"
    
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )
    return embeddings

embedding = download_embeddings()


In [25]:
embedding.model_kwargs


{'device': 'cuda'}

In [10]:
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec
from pinecone import Pinecone
index_name = "research-assistant"

pc = Pinecone()
# create the index if it doesn't exist
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=768,  # replace with your embedding dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# connect to the index
index = pc.Index(index_name)

In [11]:
from langchain_pinecone import PineconeVectorStore

PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name= index_name
)

# AI Agents creation

In [26]:
from langchain_pinecone import PineconeVectorStore

# embbeded each chunk and upsert the embeddings into your pinecone index
index_name = "research-assistant"

vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [27]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool

# RAG tool
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information from internal documentation or PDFs.

    Use this tool when the user asks about technical or implementation details
    within the project. It performs a semantic similarity search in the
    document embeddings to extract the most relevant passages.

    Example queries:
    - "Which library was used for VietorisRips?"
    - "Was DVC used for reproducibility?"
    - "Explain how GPU support was integrated with giotto-tda."
    """
    retrieved_docs = vector_store.similarity_search(query, k=2)
    if not retrieved_docs:
        return "No relevant context found in the documentation.", []
    serialized = "\n\n".join(
        f"📘 **Source:** {doc.metadata}\n{doc.page_content}" for doc in retrieved_docs
    )
    return serialized, retrieved_docs
# DuckDuckGo tool
@tool
def duckduckgo_search(query: str) -> str:
    """Search the web using DuckDuckGo.

    Use this tool for general information retrieval from online sources such as news,
    research sites, and Wikipedia. Limit results to the top 3 most relevant hits.

    Example queries:
    - "Recent research on persistent homology"
    - "Farms cultivating lettuce near Aguascalientes"
    """
    results = DuckDuckGoSearchRun().run(query)
    if not results:
        return "No results found."
    top_results = results[:3]
    return "\n".join(f"🔗 {r['title']}: {r['href']}" for r in top_results)

# Crear el client con Taviliy MCP
client = MultiServerMCPClient(
    {
        "tavily": {
            "transport": "streamable_http",
            "url": "https://mcp.tavily.com/mcp",
            "headers": {"Authorization": f"Bearer {TAVILY_API_KEY}"}
        }
    }
)
# Obtener todas las tools
tools_mcp = await client.get_tools()


In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

context_prompt = (
    "You are a specialized **retrieval assistant** focused on providing accurate and contextually relevant "
    "answers from local project documentation, PDFs, or embedded sources. "
    "You have access to a retrieval tool that searches within vectorized documents. "
    "When answering, use this tool to ground your responses in factual information extracted from the data. "
    "If you cannot find relevant context, clearly state that no related information was found. "
    "Keep answers **brief, concise, technical, and well-structured**, using bullet points when helpful. "
    "Include all relevant details necessary for a complete answer."
)
model_context = ChatOpenAI(
    model="gpt-5-nano",
    reasoning_effort='low'
)

context_agent = create_agent(
    model=model_context,
    tools=[retrieve_context],
    system_prompt=context_prompt,
)

In [29]:
query = "What MLOPS tools were used?"

for step in context_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_D7BUwxF43zJ3GSaAFcuNmvtu)
 Call ID: call_D7BUwxF43zJ3GSaAFcuNmvtu
  Args:
    query: What MLOPS tools were used?
================================= Tool Message =================================
Name: retrieve_context

📘 **Source:** {'source': 'flask_app/data_RAG/Análisis Topológico con GPU y Visualización Interactiva Basada en GenAI.pdf'}
Análisis Topológico con GPU y Visualización Interactiva 
Basada en GenAI 
 
Abstract 
Este trabajo presenta un flujo técnico reproducible para el Análisis Topológico de Datos 
(TDA) enfocado en la caracterización y comparación de municipios mediante indicadores 
climáticos, edafológicos y de uso de suelo. La propuesta integra cómputo acelerado por 
GPU a través de una adaptación de ripser++ como backend de alto rendimiento, lo que 
permite reducir significativamente el tiempo de cálculo de homología persistente dentro 
del complejo de 

In [30]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

web_prompt = (
    "You are a **web research assistant** specialized in finding relevant and reliable information from the Internet. "
    "You have access to multiple web tools (e.g., DuckDuckGo, Tavily, Wikipedia MCP). "
    "Use these tools whenever necessary to support your answer with external references. "
    "If the search results do not fully address the user's question, end your response with the sentence: "
    "'The data obtained might not answer the user's question entirely.'. "
    "Focus on clarity, conciseness, and factual accuracy, and explain the answer as briefly as possible. "

)
model_web = ChatOpenAI(
    model="gpt-5-nano",
    reasoning_effort='low'
)

web_agent = create_agent(
    model=model_web,
    tools=[duckduckgo_search]+tools_mcp,
    system_prompt=web_prompt,
)

In [31]:
query = "Who was JFK?"

for step in web_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================

John Fitzgerald Kennedy (JFK) was the 35th President of the United States, serving from 1961 until his assassination in 1963.

Key points:
- Party: Democrat
- Region: Massachusetts, U.S. Senator before becoming president
- Background: Naval officer in World War II; author and public figure
- Major themes: The New Frontier (his domestic agenda), civil rights momentum, and Cold War crises
- Notable events: Cuban Missile Crisis (1962), Bay of Pigs invasion (1961), space program momentum (Space Race)
- Assassination: Killed on November 22, 1963, in Dallas, Texas

If you meant a different JFK (e.g., John F. Kennedy Jr. or another person with those initials), tell me and I can provide details.


In [32]:
@tool
def context_retriever(request: str) -> str:
    """Retrieve answers from local project documentation."""
    result = context_agent.invoke({"messages": [{"role": "user", "content": request}]})
    response = result["messages"][-1] if isinstance(result, dict) else result
    return response.content if hasattr(response, "content") else str(response)


@tool
def web_retriever(request: str) -> str:
    """Retrieve information from the web using Tavily, DuckDuckGo, or Wikipedia MCP."""
    result = web_agent.invoke({"messages": [{"role": "user", "content": request}]})
    response = result["messages"][-1] if isinstance(result, dict) else result
    return response.content if hasattr(response, "content") else str(response)

In [34]:
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI

supervisor_prompt = (
    "You are an intelligent orchestrator responsible for coordinating specialized agents and tools. "
    "Before taking action, check the available context and metadata to determine whether the user's question is already fully answered. "
    "If sufficient context exists, respond directly without using tools. "
    "Otherwise, select the appropriate tools: use context_retriever for project or document-related questions, "
    "and web_retriever for external or general knowledge. "
    "Plan and execute multi-step tasks logically, integrating tool outputs into a single, coherent answer. "
    "Always be concise, brief, factual, and focused on the user’s intent."
)
model_supervisor = ChatOpenAI(
    model="gpt-5-nano",
    reasoning_effort='medium'
)
checkpointer = InMemorySaver()
summarization_prompt = (
    "You are a summarization assistant. "
    "Read the recent conversation messages and condense them into a brief, clear summary without losing too much info. "
    "Focus on the key facts, context, and questions that are relevant to answering the user. "
    "Avoid unnecessary details or repetitions. "
    "Format the summary in a structured way so that it can be easily consumed by another agent. "
    "Keep it factual, and as precise as possible."
)
# Inicializa el LLM
model_summary = ChatOpenAI(
    model="gpt-5-nano",
    reasoning_effort='low')

supervisor_agent = create_agent(
    model="gpt-5-nano",
    tools=[context_retriever, web_retriever],
    system_prompt=supervisor_prompt,
    middleware=[
        SummarizationMiddleware(
            model=model_summary,          # usa un modelo más ligero para resumir ####PROBAR CON "gpt-4-nano", creo que es más rápido
            max_tokens_before_summary=2000,  # resume cuando pasa los 2000 tokens
            messages_to_keep=2, #mantiene solo los últimos 2 mensajes
            summary_prompt=summarization_prompt)
    ],
    checkpointer=checkpointer,
)

In [36]:
import re

def clean_text(text: str) -> str:
    """
    Normaliza un string:
    - Convierte a minúsculas
    - Quita símbolos innecesarios (excepto ? y !)
    - Elimina espacios múltiples y caracteres innecesarios
    """
    # Convertir a minúsculas
    text = text.lower()
    
    # Quitar símbolos que no sean letras, números, espacios, ?, !
    text = re.sub(r"[^a-z0-9áéíóúüñ\s?!.,;:%+*'\"()\[\]-]", "", text, flags=re.IGNORECASE)
    
    # Reemplazar múltiples espacios por uno solo
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [37]:
texto = "¡Hola! ¿Cómo estás? Esto es un test: 100%*4 funcional."
print(clean_text(texto))

hola! cómo estás? esto es un test: 100%*4 funcional.


In [38]:
query = "Cómo se calcularon los diagramas de persistencia en el proyecto?"
query = clean_text(query)
config = make_config_from_query(query)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config=config,
):
    for update in step.values():
        if not update:  # 👈 Evita errores con None
            continue
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  context_retriever (call_g8cEI8GVdEDa5yGdxgBgDf0B)
 Call ID: call_g8cEI8GVdEDa5yGdxgBgDf0B
  Args:
    request: project persistence diagrams: how were they calculated?
================================= Tool Message =================================
Name: context_retriever

- Method: Persistent homology computed via Ripser++ (GPU-accelerated version).
- Pipeline: Integrated within giotto-tda workflow; results fed into persistence diagrams.
- Matrix/type: Based on a distance-based filtration (Vietoris–Rips) as implemented by Ripser++.
- Rationale: Ripser++ offered substantial time reductions while remaining compatible with the giotto-tda ecosystem.
- Additional context: The approach sits inside a broader TDA+GPU reproducibility framework described in the document.
================================== Ai Message ==================================

Se calcularon así:

- Método: Persistent homology c

In [ ]:
from flask import session
import uuid

if "user_id" not in session:
    session["user_id"] = str(uuid.uuid4())
user_id = session["user_id"]

In [11]:
from langchain_openai import ChatOpenAI
import inspect

# Inicializa el LLM
llm = ChatOpenAI(
    model="gpt-5-nano",
    reasoning_effort='low'
)



In [12]:
print(llm.reasoning_effort)

low


In [14]:
from langchain.agents.middleware import SummarizationMiddleware
summarization_prompt = (
    "You are a summarization assistant. "
    "Read the recent conversation messages and condense them into a brief, clear summary. "
    "Focus on the key facts, context, and questions that are relevant to answering the user. "
    "Avoid unnecessary details or repetitions. "
    "Format the summary in a structured way so that it can be easily consumed by another agent. "
    "Keep it short, factual, and precise."
)
SummarizationMiddleware(
            model=llm,          # usa un modelo más ligero para resumir ####PROBAR CON "gpt-4-nano", creo que es más rápido
            max_tokens_before_summary=2000,  # resume cuando pasa los 2000 tokens
            messages_to_keep=2, #mantiene solo los últimos 2 mensajes
            summary_prompt=summarization_prompt
        )
